SERVIR SAR TRAINING WORKSHOP 

Focus: Forest Applications 

PART 3: Time Series Change Detection

Author: Josef Kellndorfer, Ph.D., President and Senior Scientist, Earth Big Data, LLC

Revision date: January 2018


This workbook is available as a Jupyter Notebook for interactive studying of the material. This section focuses on change detection with time series analysis of SAR backscatter curves. Depending on the study region, data sets can be swapped out and subsets in image data can be selected per a user's choice. The underlying data sets assume that data are pre-processing in GDAL compatible open source formats as multi-layer time series data stacks accompanied by a date file that gives an acquisition date for each layer in a simple format with "YYYYMMDD" entries per line for each band. For the course data were prepared with Earth Big Data LLC's Sentinel-1 time series processor which includes radiometric terrain correction and multi-temporal speckle noise reduction (See [earthbigdata.com](http://earthbigdata.com), [Earth Big Data's openSAR on github](https://github.com/EarthBigData/)). Data are organized as a stack of GeoTIFF bands combined via GDAL's Virtual Raster Table (VRT) format. (See [gdal.org](http://gdal.org))

All software code in this chapter is implemented in python 3 and should execute equally on Windows, Linux or MacOS operating systems via Jupyter notebook servers. (See [jupyter.org][http://jupyter.org/))


In [ ]:
# Importing relevant python packages
import pandas as pd
import gdal
import numpy as np
import time,os

# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches

font = {'family' : 'monospace',
          'weight' : 'bold',
          'size'   : 18}
plt.rc('font',**font)

# Acquire Data

## West Africa - Biomass Site

In [ ]:
# datadirectory='/Users/josefk/Dropbox//projects/c401servir/wa/BIOsS1/'
# datefile='S32631X398020Y1315440sS1_A_vv_0001_mtfil.dates'
# imagefile='S32631X398020Y1315440sS1_A_vv_0001_mtfil.vrt'

## West Africa - Niamey Deforestation Site

In [ ]:
datadirectory='/Users/josefk/Dropbox/projects/c401servir/wa/cra/'
datefile='S32631X402380Y1491460sS1_A_vv_0001_A_mtfil.dates'
imagefile='S32631X402380Y1491460sS1_A_vv_0001_A_mtfil.vrt'

In [ ]:
# Switch to the data directory
os.chdir(datadirectory)

## Acquisition Dates

Read from the Dates file the dates in the time series and make a pandas date index

In [ ]:
dates=open(datefile).readlines()
tindex=pd.DatetimeIndex(dates)
# From the index we make and print a lookup table for 
# band numbers and dates 
datelut = []
b=1
for i in tindex:
    datelut.append(str(b)+' '+str(i.date()))
    b += 1 
for i in range(len(datelut)):
    print(datelut[i],end="  ")
    if i%4 == 0: print('\n')

## Image data

In [ ]:
rasterstack=gdal.Open(imagefile).ReadAsArray()

# Data Pre-Processing 
## Plot the global means of the Time Series 
1. conversion to power
2. compute means
3. convert to dB
4. make a pandas time series 
5. plot time series of means

In [ ]:
# 1. Conversion to Power
caldB=-83
calPwr = np.power(10.,caldB/10.)
rasterstack_pwr = np.power(rasterstack,2.)*calPwr
# 2. Compute Means
rs_means_pwr = np.mean(rasterstack_pwr,axis=(1,2))
# 3. Convert to dB
rs_means_dB = 10.*np.log10(rs_means_pwr)

In [ ]:
# 4. Make a pandas time series object
ts = pd.Series(rs_means_dB,index=tindex)

In [ ]:
# 5. Use the pandas plot function of the time series object to plot
# Put band numbers as data point labels
plt.figure(figsize=(16,8))
ts.plot()
xl = plt.xlabel('Date')
yl = plt.ylabel('$\overline{\gamma^o}$ [dB]')
for xyb in zip(tindex,rs_means_dB,range(1,len(rs_means_dB)+1)):
    plt.annotate(xyb[2],xy=xyb[0:2])


### EXERCISE
Look at the global means plot and determine from the tindex array at which dates you see  maximum and minimum values. Are relative peaks associated with seasons?

# Generate Time Series for  Point Locations or Subsets 

In python we can use the matrix slicing rules (Like Matlab) to obtain subsets of the data. For example to pick one pixel at a line/pixel location and obtain all band values, use:

>  [:,line,pixel] notation. 

Or, if we are interested in a subset at a offset location we can use:

> [:,yoffset:(yoffset+yrange),xoffset:(xoffset+xrange)]

In the section below we will learn how to generate time series plots for point locations (pixels) or areas (e.g. a 5x5 window region). To show  individual bands, we define a showImage function which incorporates the matrix slicing from above.

In [ ]:
def showImage(rasterstack,tindex,bandnbr,subset=None,vmin=None,vmax=None):
    '''Input: 
    rasterstack stack of images in SAR power units
    tindex time series date index
    bandnbr bandnumber of the rasterstack to dissplay'''
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # If vmin or vmax are None we use percentiles as limits:
    if vmin==None: vmin=np.percentile(rasterstack[bandnbr-1].flatten(),5)
    if vmax==None: vmax=np.percentile(rasterstack[bandnbr-1].flatten(),95)

    ax1.imshow(rasterstack[bandnbr-1],cmap='gray',vmin=vmin,vmax=vmax)
    ax1.set_title('Image Band {} {}'.format(bandnbr,tindex[bandnbr-1].date()))
    if subset== None:
        bands,ydim,xdim=rasterstack.shape
        subset=(0,0,xdim,ydim)
        
    ax1.add_patch(patches.Rectangle((subset[0],subset[1]),subset[2],subset[3],fill=False,edgecolor='red'))
    ax1.xaxis.set_label_text('Pixel')
    ax1.yaxis.set_label_text('Line')
    
    ts_pwr=np.mean(rasterstack[:,subset[1]:(subset[1]+subset[3]),
                       subset[0]:(subset[0]+subset[2])],axis=(1,2))
    ts_dB=10.*np.log10(ts_pwr)
    ax2.plot(tindex,ts_dB)
    ax2.yaxis.set_label_text('$\gamma^o$ [dB]')
    ax2.set_title('$\gamma^o$ Backscatter Time Series')
    # Add a vertical line for the date where the image is displayed
    ax2.axvline(tindex[bandnbr-1],color='red')

    fig.autofmt_xdate()

### Exercise
Compare band 24 and band 43 visually

In [ ]:
bandnbr=24  # 
subset=[5,20,3,3]
# subset=[30,15,3,3]
# subset=[12,10,3,3]

In [ ]:
showImage(rasterstack_pwr,tindex,bandnbr,subset)

In [ ]:
bandnbr=43
showImage(rasterstack_pwr,tindex,bandnbr,subset)

### EXERCISE
For subset (5,20,3,3):
1. What are the dates where backscatter falls below - 11 dB?
2. Compute the  gradients (backscatter difference between two consecutive dates. 
3. What is the largest gradient of backscatter drop between two consecutive dates? 
4. What are the dates associated with this gradient (before and after)?



### Helper function the generate a time series object

In [ ]:
def timeSeries(rasterstack_pwr,tindex,subset):
    # Extract the means along the time series axes
    # raster shape is time steps, lines, pixels. 
    # With axis=1,2, we average lines and pixels for each time 
    # step (axis 0)
    ts_pwr=np.mean(rasterstack_pwr[:,subset[1]:(subset[1]+subset[3]),
                       subset[0]:(subset[0]+subset[2])],axis=(1,2))
    # convert the means to dB
    ts_dB=10.*np.log10(ts_pwr)
    # make the pandas time series object
    ts = pd.Series(ts_dB,index=tindex)
    # return it
    return ts

Using the timeSeries(...) function to make a time series object for the chosen subset:

In [ ]:
ts = timeSeries(rasterstack_pwr,tindex,subset)

Plot the object:

In [ ]:
_=ts.plot(figsize=(16,4))  # _= is a trick to suppress more output.

### ENTER YOUR CODE HERE

In [ ]:
# 1. What are the dates where backscatter falls below - 11 dB?


In [ ]:
# 2. Compute the gradients (backscatter difference 
# between two consecutive dates.


In [ ]:
# 3. What is the largest gradient of backscatter drop 
# between two consecutive dates? 


In [ ]:
# What are the dates associated with this gradient 
# (before and after)


### Solutions:

In [ ]:
# 1. 
ts[ts<-11].index

In [ ]:
# 2.
gradient_lag1 = ts.diff(1)
gradient_lag1.plot()

In [ ]:
# 3. 
gradient_lag1.min()

In [ ]:
# 4.
gradient_lag1[gradient_lag1==gradient_lag1.min()]

In [ ]:
before = gradient_lag1[gradient_lag1==gradient_lag1.min()].index[0]
before

In [ ]:
after=tindex[tindex>before][0]
after

### Question: Can you field verify that change occured at this location between these two dates?

## Seasonal Subsets of time series records
Let's expand upon SAR time series analysis. Often it is desirable to subset time series by season or months to compare with similar conditions of a previous year's observation. For example, in analyzing C-Band backscatter data, it might be useful to limit comparative analysis to dry season observations only as soil moisture might confuse signals during the wet seasons. In this section we will expand upon the concepts of subsetting time series along the time axis. We will make use of the pandas datatime index tools

- month
- day of year 

First we extract the time series again for a hectare the pixel at the subset location (5,20,5,5). We then convert the pandas time series to a pandas DataFrame to allow for more processing options. We also label the data value column as 'g0' for gamma0:

In [ ]:
subset=(5,20,5,5)
ts = timeSeries(rasterstack_pwr,tindex,subset)
tsdf = pd.DataFrame(ts,index=ts.index,columns=['g0'])

# Plot
ylim=(-20,-5)
tsdf.plot(figsize=(16,4))
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: 5,20,5,5  ')
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_=plt.legend(["C-VV Time Series"])

### Start the time series in November 2015 
We can use the pandas index parameters like month to make seasonal subsets

In [ ]:
tsdf_sub1=tsdf[tsdf.index>'2015-11-01']

# Plot
tsdf_sub1.plot(figsize=(16,4))
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'.format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_=plt.legend(["C-VV Time Series"])

### Subset by months: 

We can make use of pandas DateTimeIndex object **index.month** and numpy's **logical_and** function to subset a time series easily by month.

####  March to May data only

In [ ]:
tsdf_sub2=tsdf_sub1[
    np.logical_and(tsdf_sub1.index.month>=3,tsdf_sub1.index.month<=5)]

# Plot
fig, ax = plt.subplots(figsize=(16,4))
tsdf_sub2.plot(ax=ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'
          .format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_=plt.legend(["March-May"])

####  All other months
Using numpy's boolean **invert** function, we can invert a selection and in this example get to all other months:

In [ ]:
tsdf_sub3=tsdf_sub1[np.invert(
    np.logical_and(tsdf_sub1.index.month>=3,tsdf_sub1.index.month<=5))]

# Plot
fig, ax = plt.subplots(figsize=(16,4))
tsdf_sub3.plot(ax=ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'
          .format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_=plt.legend(["June-February"])

###  Group time series by Year to compare average backscatter values


In [ ]:
ts_sub_by_year = tsdf_sub1.groupby(pd.Grouper(freq="Y"))

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
for label, df in ts_sub_by_year:
    df.g0.plot(ax=ax, label=label.year)
plt.legend()
# ts_sub_by_year.plot(ax=ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'
          .format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)


### Make a pivot table to group year and sort by day of year for plotting overlapping time series
First we add two coluns to the data frame:
- day of year (doy)
- year

In [ ]:
# Add doy
tsdf_sub1 = tsdf_sub1.assign(doy=tsdf_sub1.index.dayofyear)
# Add year
tsdf_sub1 = tsdf_sub1.assign(year=tsdf_sub1.index.year)

Then a pivot table gets created which has day of year as the index and years as columns:

In [ ]:
piv=pd.pivot_table(tsdf_sub1,index=['doy'],columns=['year'],values=['g0'])
# Set the names for the column indices
piv.columns.set_names(['g0','Year'],inplace=True) 
print(piv.head(10))
print('...\n',piv.tail(10))

In [ ]:
piv.columns.set_names(['g0','year'],inplace=True)

As we can see, there are NaN values on the days in a year where no acquisition took place. Now we use time weighted interpolation to fill the dates for all the observations in any given year. For **time weighted interpolation** to work we need to create a dummy year as a date index, perform the interplation, and reset the index to the day of year. This is accomplished with the following steps:

In [ ]:
# Add fake dates for year 100 to enable time sensitive interpolation 
# of missing values in the pivot table
year_doy = ['2100-{}'.format(x) for x in piv.index]
y100_doy=pd.DatetimeIndex(pd.to_datetime(year_doy,format='%Y-%j'))

# make a copy of the piv table and add two columns
piv2=piv.copy()
piv2=piv2.assign(d100=y100_doy) # add the fake year dates
piv2=piv2.assign(doy=piv2.index) # add doy as a column to replace as index later again

# Set the index to the dummy year
piv2.set_index('d100',inplace=True,drop=True)

# PERFORM THE TIME WEIGHTED INTERPOLATION
piv2 = piv2.interpolate(method='time')  # TIME WEIGHTED INTERPOLATION!

# Set the index back to day of year.
piv2.set_index('doy',inplace=True,drop=True)

Let's inspect the new pivot table and see wheather we interpolated the NaN values where it made sense:

In [ ]:
print(piv2.head(10))
print('...\n',piv2.tail(10))

Now we can plot the time series data with overlapping years 

In [ ]:
piv2.plot(figsize=(16,8))
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile,\
Subset: 5,20,5,5  ')
plt.ylabel('$\gamma^o$ [dB]')
plt.xlabel('Day of Year')
_=plt.ylim(ylim)

# Change Detection on the Time Series Data
The goal being change detection we can now analyze the time series data for change. We will discuss two approaches:
1. Year-to-year differencing of the subsetted time series
2. Cumulative Sum based change detection

In [ ]:
# Difference between years
# Set a dB change threshold
thres=3

In [ ]:
diff1716 = (piv2.g0[2017]-piv2.g0[2016])

## Year-to-Year Change Detection

We compute the differences between the interpolated time series and look for change with a threshold value.

In [ ]:
_=diff1716.plot('line')

In [ ]:
thres_exceeded = diff1716[abs(diff1716) > thres]
thres_exceeded

From the three_exceeded dataframe we can infer the first date at which the threshold was exeeded. We would label that as a change point. As an additional criteria for labeling a change point, one can also consider the number of observations after identification of a change point where backscatter differed from the year before. If only one or two observations differed from the year before this could be considered an outlier. Addtionally, one can introduce smoothing operations with the interpolation, e.g. a 


### EXERCISE:
Work through the workbook again with selection of a different point and determine if it is a change point.



## Cumulative Sums for Change Detection
Another approach to detect change in regularly acquired data is employing cumulative sums. Changes are determined against mean observations of time series. A full explanation and examples from the the financial sector can be found at [http://www.variation.com/cpa/tech/changepoint.html](http://www.variation.com/cpa/tech/changepoint.html)

### Time Series and means
First let's consider a time series and it's mean observation. 
We look at two full years of observations from Sentinel-1 data for an area where we suspect change. In the following we consider $X$ as a time series

$X = (X_1,X_2,...,X_n)$

with

- $X_i$  SAR backscatter at time $i=1,...,n$
- $n$ number of observations in the time series

In [ ]:
subset=(5,20,3,3)
#subset=(12,5,3,3)
ts1 = timeSeries(rasterstack_pwr,tindex,subset)
X = ts1[ts1.index>'2015-10-31']

## Filtering the time series for outliers
It is advantagous in noisy SAR time series data like C-Band data to filter  on the time axis. Pandas offers a "rolling" function for these purposes. With that function we can choose for example a median filter along the time axis. Below is an example of you a median filter for an observation filters the time series when the observation before and after a time stamps are part of the filter. 

 

In [ ]:
Xr=X.rolling(5,center=True).median()
Xr.plot()
_=X.plot()

Let's plot the time series and it's mean over the time span

In [ ]:
X=Xr  # Uncomment if rolling mean is wanted for further computation
Xmean = X.mean()

In [ ]:
fig,ax=plt.subplots(figsize=(16,4))
X.plot()
plt.ylabel('$\gamma^o$ [dB]')
ax.axhline(Xmean,color='red')
_=plt.legend(['$\gamma^o$','$\overline{\gamma^o}$'])

Let's determind the residuals of the time series against the mean'

$R = X_i -\overline{X}$

In [ ]:
R = X - Xmean

Now we compute the cumulative sum of the residuals and plot it:

$S = \displaystyle\sum_1^n{R_i}$

In [ ]:
S = R.cumsum()

_=S.plot(figsize=(16,8))

An estimator for the magnitude of change is given as the difference between the maximum and minimum value of S

$S_{DIFF} = S_{MAX} - S_{MIN}$

In [ ]:
Sdiff=S.max() - S.min()
Sdiff

A candidate change point is identified from the $S$ curve at the time where $S_{MAX}$ is found:

$T_{{CP}_{before}} = T(S_i = S_{MAX})$

with

- $T_{{CP}_{before}}$ Timestamp of last observation before change
- $S_i$    Cumulative Sum of R with $i=1,...n$
- $n$      Number of observations in the time series 

The first observation after change occured ($T_{{CP}_{after}}$) is then found as the first observation in the time series following $T_{{CP}_{before}}$.

For our example time series X these points are:

In [ ]:
t_cp_before = S[S==S.max()].index[0]
print('Last date before change: {}'.format(t_cp_before.date()))

In [ ]:
t_cp_after = S[S.index > t_cp_before].index[0]
print('First date after change: {}'.format(t_cp_after.date()))

## Bootstrapping the cumulative sums by randomly reordering the time series

We can determine if an identified change point is indeed a valid detection by randomly reordring the time series and comparing the various S curves. During bootstrapping we count how many times the $S_{DIFF}$ values are greater than $S_{{DIFF}_{random}}$ of the identified change point. A confindence level $CL$ is computed as

$CL = \frac{N_{GT}}{N_{bootstraps}}$

with 

- $N_{GT}$    Number of times $S_{DIFF}$ > $S_{{DIFF}_{random}}$  
- $N_{bootstraps}$    Number of bootstraps randomizing $R$

Another metric for the significance of a change point is 1 minus the ratio of the mean of the $S_{{DIFF}_{random}}$ values and $S_{DIFF}$. The closer this value is approaching 1, the more significant the change point: 

$CP_{significance} = 1 - \left( \frac{\sum_{b=1}^{N_{bootstraps}}{S_{{DIFF}_{{random}_i}}}}{N_{bootstraps}} \middle/ S_{DIFF} \right)$

The python code to conduct the boot strapping, including visualization of the S curves is below:

In [ ]:
n_bootstraps=500  # bootstrap sample size
fig,ax = plt.subplots(figsize=(16,8))
S.plot(ax=ax,linewidth=3)
ax.set_ylabel('Cumulative Sums of the Residuals')
fig.legend(['S Curve for Candidate Change Point'],loc=3)
Sdiff_random_sum=0
Sdiff_random_max=0  # to keep track of the maxium Sdiff of the 
               # bootstrapped sample
n_Sdiff_gt_Sdiff_random=0  # to keep track of the maxium Sdiff of the 
               # bootstrapped sample
for i in range(n_bootstraps):
    Rrandom = R.sample(frac=1)  # Randomize the time steps of the residuals
    Srandom = Rrandom.cumsum()
    Sdiff_random=Srandom.max()-Srandom.min()
    Sdiff_random_sum += Sdiff_random
    if Sdiff_random > Sdiff_random_max:
        Sdiff_random_max = Sdiff_random
    if Sdiff > Sdiff_random:
        n_Sdiff_gt_Sdiff_random += 1
    Srandom.plot(ax=ax)
_=ax.axhline(Sdiff_random_sum/n_bootstraps)

In [ ]:
CL = 1.*n_Sdiff_gt_Sdiff_random/n_bootstraps
print('Confidence Level for change point {} percent'.format(CL*100.))

In [ ]:
CP_significance = 1. - (Sdiff_random_sum/n_bootstraps)/Sdiff 
print('Change point significance metric: {}'.format(CP_significance))

Another useful metric to determine strength of a change point is the normalized integral $S_{ni}$ of the absolute values of the S curve:

$S_{normintegral} = \frac{\int_{i=1}^n \frac{abs(S_i)}{\max{abs(S)}} } {n}$

In [ ]:
# NaN's to be excluded in the computation 
S_ni=(S.abs()/S.abs().max()).cumsum().max()/len(S[S != np.nan])
print('Normalized Integral of cumulative sum: {}'.format(S_ni))

### EXERCISE
Conduct the change point analysis for different subsets in the traning data

## Selection of threshold values
$CL$ and $CP_{significance}$ can be used as threshold values for the acceptance or rejection of a candidate threshold. These values are to some degree specific to a SAR sensor and environmental conditions. E.g. L-Band SAR has a more pronounced decrease in backscatter after forest disturbance and logging, whereas C-Band can have more ambigious signals. Also moisture regime changes, e.g. with snow cover, freeze/thaw conditions or dry/wet season changes have an influence on the time series signal. For example El Nino years can suggest changes solely due to different wetting and dryup conditions pertinent to a particular year. For this reason other techniques  can be added to the SAR time series ananlysis. Two techniques can readily be thought of:

- Subsetting of time series by seasons
- Detrending time series with global image means

If year-to-year comparison is the focus, the first approach likely leads to subsets that are too small for meaningful cumulative sum change point detection. The approach of interannual differencing as discussed above likely performs better.


In the following we explore the approach to detrend the data with global image means.

### De-trending time series with global image means
The idea of de-trending time series with global image meams should prepare time series for a somewhat robuster change point detection as global image time series anomalies stemming calibration or seasonal trends are removed prior to time series analysis. This de-trending needs to be performed with large subsets so real change is not influencing the image statistics. 

NOTE: For our small subset, we will see  some of these effects.

Let's start by building a global image means time series:

In [ ]:
means_pwr = np.mean(rasterstack_pwr,axis=(1,2))
means_dB = 10.*np.log10(means_pwr)
gm_ts = pd.Series(means_dB,index=tindex)
gm_ts=gm_ts[gm_ts.index > '2015-10-31']  # filter dates
gm_ts=gm_ts.rolling(5,center=True).median()

In [ ]:
gm_ts.plot()

In [ ]:
X.plot()

In [ ]:
Xd=X-gm_ts
Xmean=Xd.mean()
Xd.plot()

In [ ]:
R = Xd - Xmean

Now we compute the cumulative sum of the residuals and plot it:

$S = \displaystyle\sum_1^n{R_i}$

In [ ]:
S = R.cumsum()

_=S.plot(figsize=(16,8))

An estimator for the magnitude of change is given as the difference between the maximum and minimum value of S

$S_{DIFF} = S_{MAX} - S_{MIN}$

In [ ]:
Sdiff=S.max() - S.min()
Sdiff

A candidate change point is identified from the $S$ curve at the time where $S_{MAX}$ is found:

$T_{{CP}_{before}} = T(S_i = S_{MAX})$

with

- $T_{{CP}_{before}}$ Timestamp of last observation before change
- $S_i$    Cumulative Sum of R with $i=1,...n$
- $n$      Number of observations in the time series 

The first observation after change occured ($T_{{CP}_{after}}$) is then found as the first observation in the time series following $T_{{CP}_{before}}$.

For our example time series X these points are:

In [ ]:
t_cp_before = S[S==S.max()].index[0]
print('Last date before change: {}'.format(t_cp_before.date()))

In [ ]:
t_cp_after = S[S.index > t_cp_before].index[0]
print('First date after change: {}'.format(t_cp_after.date()))

## Bootstrapping the cumulative sums by randomly reordering the time series

We can determine if an identified change point is indeed a valid detection by randomly reordring the time series and comparing the various S curves. During bootstrapping we count how many times the $S_{DIFF}$ values are greater than $S_{{DIFF}_{random}}$ of the identified change point. A confindence level $CL$ is computed as

$CL = \frac{N_{GT}}{N_{bootstraps}}$

with 

- $N_{GT}$    Number of times $S_{DIFF}$ > $S_{{DIFF}_{random}}$  
- $N_{bootstraps}$    Number of bootstraps randomizing $R$

Another metric for the significance of a change point is 1 minus the ratio of the mean of the $S_{{DIFF}_{random}}$ values and $S_{DIFF}$. The closer this value is approaching 1, the more significant the change point: 

$CP_{significance} = 1 - \left( \frac{\sum_{b=1}^{N_{bootstraps}}{S_{{DIFF}_{{random}_i}}}}{N_{bootstraps}} \middle/ S_{DIFF} \right)$

The python code to conduct the boot strapping, including visualization of the S curves is below:

In [ ]:
n_bootstraps=500  # bootstrap sample size
fig,ax = plt.subplots(figsize=(16,8))
S.plot(ax=ax,linewidth=3)
ax.set_ylabel('Cumulative Sums of the Residuals')
fig.legend(['S Curve for Candidate Change Point'],loc=3)
Sdiff_random_sum=0
Sdiff_random_max=0  # to keep track of the maxium Sdiff of the 
               # bootstrapped sample
n_Sdiff_gt_Sdiff_random=0  # to keep track of the maxium Sdiff of the 
               # bootstrapped sample
for i in range(n_bootstraps):
    Rrandom = R.sample(frac=1)  # Randomize the time steps of the residuals
    Srandom = Rrandom.cumsum()
    Sdiff_random=Srandom.max()-Srandom.min()
    Sdiff_random_sum += Sdiff_random
    if Sdiff_random > Sdiff_random_max:
        Sdiff_random_max = Sdiff_random
    if Sdiff > Sdiff_random:
        n_Sdiff_gt_Sdiff_random += 1
    Srandom.plot(ax=ax)
_=ax.axhline(Sdiff_random_sum/n_bootstraps)

In [ ]:
CL = n_Sdiff_gt_Sdiff_random/n_bootstraps
print('Confidence Level for change point {} percent'.format(CL*100.))

In [ ]:
CP_significance = 1. - (Sdiff_random_sum/n_bootstraps)/Sdiff 
print('Change point significance metric: {}'.format(CP_significance))

Another useful metric to determine strength of a change point is the normalized integral $S_{ni}$ of the absolute values of the S curve:

$S_{normintegral} = \frac{\int_{i=1}^n \frac{abs(S_i)}{\max{abs(S)}} } {n}$

In [ ]:
S_ni=(S.abs()/S.abs().max()).cumsum().max()/len(S[S != np.nan])
print('Normalized Integral of cumulative sum: {}'.format(S_ni))

## Cumulative Sum Change Detection for the entire image
With numpy arrays we can apply the concept of cumulative sum change detection analysis effectively on the entire image stacke. We take advantage of array slicing and axis-based computing in numpy. axis 0 is the time time domain in out raster stacks

In [ ]:
# Test this also in the dB scale We'll work in the dB scale
X = rasterstack_pwr
# Filter out the first layer ( Dates >= '2015-11-1')
X_sub=X[1:,:,:]
tindex_sub=tindex[1:]
X= 10.*np.log10(X_sub)  # Uncomeent to test dB scale 

In [ ]:
plt.figure()
bandnbr=0
vmin=np.percentile(X[bandnbr],5)
vmax=np.percentile(X[bandnbr],95)
plt.title('Band  {} {}'.format(bandnbr+1,tindex_sub[bandnbr].date()))
plt.imshow(X[0],cmap='gray',vmin=vmin,vmax=vmax)
_=plt.colorbar()

In [ ]:
Xmean=np.mean(X,axis=0)

In [ ]:
R=X-Xmean

In [ ]:
plt.imshow(R[0])
plt.title('Redsiduals')
_=plt.colorbar()


In [ ]:
S = np.cumsum(R,axis=0)
Smax= np.max(S,axis=0)
Smin= np.min(S,axis=0)
Sdiff=Smax-Smin
fig,ax=plt.subplots(1,3,figsize=(16,4))
vmin=Smin.min()
vmax=Smax.max()
p=ax[0].imshow(Smax,vmin=vmin,vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smin,vmin=vmin,vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Sdiff,vmin=vmin,vmax=vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
_=fig.colorbar(p,cax=cbar_ax)

## Mask Sdiff with a priori threshold for expected change
If we have an assumption as to how much actual change we expect in the image, we can threshold $S_{diff}$ to reduce computiation of the bootstrapping. For land cover chaangem we would not expect more than 5-10% change in a landscape. So, if the test region is reasonably large, setting a threshold for expected change to 10% would be appropriate. Thus we can set a mask with the 90th percentile of the histogram of $S_{diff}$. In our example we'll start out with a very conservative threshold of 50%.

The histogram for $S_{diff}$ is shown below.



In [ ]:
h=plt.hist(Sdiff.flatten(),bins=50)

At the 50% percentile, the threshold value is

In [ ]:
thres=np.percentile(h[1],50)
thres

Using this threshold, we can visualize the candidate changpoints:

In [ ]:
Sdiffmask=Sdiff<thres
_=plt.imshow(Sdiffmask,cmap='gray')

Now we can filter our Residuals and perform bootstrapping analysis on these data. We make use of numpy masekd arrays for this purpose.


In [ ]:
Rmask = np.broadcast_to(Sdiffmask,R.shape)

In [ ]:
Rmasked = np.ma.array(R,mask=Rmask)

On the masked time series stack of residuals we can computed the cumulative sums:

In [ ]:
Smasked = np.ma.cumsum(Rmasked,axis=0)

$S_{MAX}$, $S_{MIN}$, $S_{DIFF}$ can also be computed on the masked arrays :

In [ ]:
plt.imshow(Rmasked.mask[0])

In [ ]:
Smasked = np.ma.cumsum(Rmasked,axis=0)
Smasked_max= np.ma.max(Smasked,axis=0)
Smasked_min= np.ma.min(Smasked,axis=0)
Smasked_diff=Smasked_max-Smasked_min
fig,ax=plt.subplots(1,3,figsize=(16,4))
vmin=Smasked_min.min()
vmax=Smasked_max.max()
p=ax[0].imshow(Smasked_max,vmin=vmin,vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smasked_min,vmin=vmin,vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Smasked_diff,vmin=vmin,vmax=vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
_=fig.colorbar(p,cax=cbar_ax)



## Bootstrapping over the masked change point candidates
We can now perform the bootstrapping analysis over the not masked out values. For efficient computing we permutate the index of the time axis.

In [ ]:
random_index=np.random.permutation(Rmasked.shape[0])
Rrandom=Rmasked[random_index,:,:]

fig,ax=plt.subplots(1,2,figsize=(8,4))
ax[0].imshow(Rmasked[0])
ax[0].set_title('Band 0')
ax[1].imshow(Rrandom[0])
_=ax[1].set_title('Band 0 Randomized')


In [ ]:
Smasked_max=np.ma.max(Smasked,axis=0)

Below is the numpy based implementation of the bootstrapping over all pixels. Note the efficient implementation using nympy masked arrays.

In [ ]:
n_bootstraps=1000  # bootstrap sample size

# to keep track of the maxium Sdiff of the bootstrapped sample:
Sdiff_random_max = np.ma.copy(Smasked_diff) 
Sdiff_random_max[~Sdiff_random_max.mask]=0
# to compute the Sdiff sums of the bootstrapped sample:
Sdiff_random_sum = np.ma.copy(Smasked_diff) 
Sdiff_random_sum[~Sdiff_random_max.mask]=0
# to keep track of the count of the bootstrapped sample
n_Sdiff_gt_Sdiff_random = np.ma.copy(Smasked_diff) 
n_Sdiff_gt_Sdiff_random[~n_Sdiff_gt_Sdiff_random.mask]=0
for i in range(n_bootstraps):
    # For efficiency, we shuffle the time axis index and use that 
    #to randomize the masked array
    random_index=np.random.permutation(Rmasked.shape[0])
    # Randomize the time step of the residuals
    Rrandom = Rmasked[random_index,:,:]  
    Srandom = np.ma.cumsum(Rrandom,axis=0)
    Srandom_max=np.ma.max(Srandom,axis=0)
    Srandom_min=np.ma.min(Srandom,axis=0)
    Sdiff_random=Srandom_max-Srandom_min
    Sdiff_random_sum += Sdiff_random
    Sdiff_random_max[np.ma.greater(Sdiff_random,Sdiff_random_max)]=\
    Sdiff_random[np.ma.greater(Sdiff_random,Sdiff_random_max)]
    n_Sdiff_gt_Sdiff_random[np.ma.greater(Smasked_diff,Sdiff_random)] += 1

Now we can compute for all pixels the confidence level $CL$, the change point significance metric $CP_{significance} and the product of the two as our confidence metric for identified changepoints.

In [ ]:
CL = n_Sdiff_gt_Sdiff_random/n_bootstraps
CP_significance = 1.- (Sdiff_random_sum/n_bootstraps)/Sdiff 
#Plot
fig,ax=plt.subplots(1,3,figsize=(16,4))
a = ax[0].imshow(CL*100)
fig.colorbar(a,ax=ax[0])
ax[0].set_title('Confidence Level %')
a = ax[1].imshow(CP_significance)
fig.colorbar(a,ax=ax[1])
ax[1].set_title('Significance')
a = ax[2].imshow(CL*CP_significance)
fig.colorbar(a,ax=ax[2])
_=ax[2].set_title('CL x S')

Now if we were to set a threshold of 0.5 for the product as identified change our change map would look like the following figure:


In [ ]:
cp_thres=0.5

In [ ]:
plt.imshow(CL*CP_significance <  cp_thres,cmap='cool')

Our last step is the idenficiaton of the change points is to extract the timing of the change. We will produce a raster layer that shows the band number of this first date after detected change. We will make use of the numpy indexing scheme. First, we create a combined mask of the first threshold and the identified change points after the bootstrapping. For this we use the numpy "mask_or" operation.


In [ ]:
# make a mask of our change points from the new threhold and the previous 
# mask
cp_mask=np.ma.mask_or(CL*CP_significance<cp_thres,CL.mask)
# Broadcast the mask to the shape of the masked S curves
cp_mask2 = np.broadcast_to(cp_mask,Smasked.shape)
# Make a numpy masked array with this mask
CPraster = np.ma.array(Smasked.data,mask=cp_mask2)

To retrieve the dates of the change points we find the band indices in the time series along the time axis where the the maximum of the cumulative sums was located. Numpy offers the "argmax" function for this purpose.

In [ ]:
CP_index= np.ma.argmax(CPraster,axis=0)
change_indices = list(np.unique(CP_index))
change_indices.remove(0)
print(change_indices)
# Look up the dates from the indices to get the change dates
alldates=tindex[tindex>'2015-10-31']
change_dates=[str(alldates[x+1].date()) for x in change_indices]
print(change_dates)

Lastly, we visualize the change dates by showing the CP_index raster and label the change dates.

In [ ]:
ticks=change_indices
ticklabels=change_dates

cmap=plt.cm.get_cmap('magma',ticks[-1])
fig, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(CP_index,interpolation='nearest',cmap=cmap)
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(p,cax=cbar_ax)

ax.set_title('Dates of Change')
# cbar = fig.colorbar(cax,ticks=ticks)
cbar=fig.colorbar(cax,ticks=ticks,orientation='horizontal')
_=cbar.ax.set_xticklabels(ticklabels,size=10,rotation=45,ha='right')  

## Secondary Change Points
After detection of a change point in the time series we can split the series in before and after change subsets. For forest degradation or deforestation detection for example this could apply when over the course of a multi-year time series selective logging preceeds a clearing event or converion of a logged plot to agriculture or regrowth, which show typically different time series profiles of radar backscatter. The approach to detect secondary change points would be to repeat analysis of the time series split into before and after change point detection. 

# Conclusion

Pandas and numpy are powerful open source scripting tools to implement change point detection on large data stacks. For image based analysis numpy offers more efficient implementations compared to pandas, whereas pandas is more powerful in date time processing, e.g. time-weighted interpolation. 